<a href="https://colab.research.google.com/github/iqbal141290/HResP/blob/main/HResP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
print('H-RESP (Heave Response Program in Random Waves)')
print('Dibuat oleh: Muhammad Iqbal, S.T., M.T., Ph.D - Teknik Perkapalan UNDIP')
print('Tanggal: 3 Juni 2025')

import numpy as np
import pandas as pd
import scipy.interpolate as interp
from scipy.integrate import trapezoid

# ==== INPUT DARI USER ====
print('\n\n=== INPUT PARAMETERS ===')
v = float(input('Kecepatan kapal (m/s): '))
L = float(input('Panjang kapal (m): '))
Hs = float(input('Tinggi Gelombang Signifikan, Hs (m): '))
Tp = float(input('Peak Period, Tp (s): '))
gama = float(input('Gamma (JONSWAP): '))
heading_deg = float(input('Heading (arah datang gelombang) dalam derajat: '))
heading = np.deg2rad(heading_deg)

# ==== IMPORT DATA RAO DARI CSV ====
file_path = input("Masukkan nama file CSV (contoh: Heave.csv): ").strip()
df = pd.read_csv(file_path, sep='\t') # Gunakan jika pemisah data berupa spasi
#df = pd.read_csv(file_path, sep=';') # Gunakan jika pemisah data berupa ;
#df = pd.read_csv(file_path) # Gunakan jika pemisah data berupa ,

wave_ratio = df.iloc[:, 0].to_numpy()
Heave_RAO = df.iloc[:, 1].to_numpy()

# ==== PROSES GEOMETRI DAN GELOMBANG ====
wave_length = wave_ratio * L
k = 2 * np.pi / wave_length
w = np.sqrt(k * 9.81)
w_e = w * (1 - (w * v * np.cos(heading)) / 9.81)
T_e = 2 * np.pi / w_e

# ==== INTERPOLASI RAO ====
no_points = len(Heave_RAO)
x = w_e
y = Heave_RAO
xx = np.linspace(min(x), max(x), no_points)
RAO = interp.CubicSpline(x, y)(xx)

# ==== SPEKTRUM GELOMBANG (JONSWAP) ====
ww = np.linspace(min(w), max(w), no_points)
we = ww * (1 - (ww * v * np.cos(heading)) / 9.81)
fp = 2 * np.pi / Tp
fac1 = (320 * Hs**2) / Tp**4
sigma = np.where(ww <= fp, 0.07, 0.09)
Aa = np.exp(-((ww / fp - 1) / (sigma * np.sqrt(2)))**2)
fac2 = ww**-5
fac31 = np.exp(-5 / 4 * (ww / fp)**-4)
fac4 = gama**Aa
S = fac1 * fac2 * fac31 * fac4
S_we = S / (1 - (2 * ww * v / 9.81) * np.cos(heading))
S_we_inter = interp.CubicSpline(we, S_we)(xx)

# ==== SPEKTRUM RESPON ====
S_res = RAO**2 * S_we_inter
velocity = xx**2 * RAO**2 * S_we_inter
acceleration = xx**4 * S_res

# ==== MOMEN DAN RMS ====
m_0 = np.trapezoid(S_res, xx)
m_2 = np.trapezoid(velocity, xx)
m_4 = np.trapezoid(acceleration, xx)

RMS_m0 = np.sqrt(m_0)
RMS_m2 = np.sqrt(m_2)
RMS_m4 = np.sqrt(m_4)

# ==== OUTPUT ====

print('\n\n=== HASIL PERHITUNGAN ===')
print(f'm0 (energi heave): {m_0:.4f}')
print(f'm2 (energi kecepatan): {m_2:.4f}')
print(f'm4 (energi percepatan): {m_4:.4f}')
print(f'RMS Heave (m): {RMS_m0:.4f}')
print(f'RMS Velocity (m/s): {RMS_m2:.4f}')
print(f'RMS Acceleration (m/s²): {RMS_m4:.4f}')


H-RESP (Heave Response Program in Random Waves)
Dibuat oleh: Muhammad Iqbal, S.T., M.T., Ph.D - Teknik Perkapalan UNDIP
Tanggal: 3 Juni 2025


=== INPUT PARAMETERS ===
Kecepatan kapal (m/s): 3.654
Panjang kapal (m): 30.977
Tinggi Gelombang Signifikan, Hs (m): 1
Peak Period, Tp (s): 3.5
Gamma (JONSWAP): 3.3
Heading (arah datang gelombang) dalam derajat: 180
Masukkan nama file CSV (contoh: Heave.csv): /content/drive/MyDrive/Colab Notebooks/Heave.csv


=== HASIL PERHITUNGAN ===
m0 (energi heave): 0.0027
m2 (energi kecepatan): 0.0107
m4 (energi percepatan): 0.0464
RMS Heave (m): 0.0522
RMS Velocity (m/s): 0.1036
RMS Acceleration (m/s²): 0.2154
